# homework_of_airbnb_PART3_MODELing

## impute data

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import datetime
from sklearn.preprocessing import *
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder

In [12]:
xtest = pd.read_csv('C:\\DAL database\Airbnb_New_User_Bookings\\homwork_of_aibnb_xtest1.csv',index_col=0)
xtrain = pd.read_csv('C:\\DAL database\Airbnb_New_User_Bookings\\homwork_of_aibnb_xtrain1.csv',index_col=0)
ytrain = pd.read_csv('C:\\DAL database\Airbnb_New_User_Bookings\\homwork_of_aibnb_ytrain1.csv',header=None)

In [3]:
print(xtrain.shape)
print(ytrain.shape)
print(xtest.shape)

(213451, 312)
(213451, 1)
(62096, 312)


In [13]:
ytrain_le = LabelEncoder().fit_transform(ytrain)
print(ytrain_le.shape)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(213451,)


## NDCG method

In [5]:
from sklearn.metrics import make_scorer

def dcg_score(y_true,y_score,k=5):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true,order[:k])
    gain = 2**y_true - 1
    discounts = np.log2(np.arange(len(y_true))+2)
    return np.sum(gain/discounts)

def ndcg_score(ground_truth,predictions,k=5):
    LabelBinarizer().fit(range(len(np.unique(ground_truth))))
    T = LabelBinarizer().fit_transform(ground_truth)
    
    scores = []
    
    for y_true,y_score in zip(T,predictions):
        actual1 = dcg_score(y_true,y_score,k)
        best = dcg_score(y_true,y_true,k)
        score = float(actual1)/float(best)
        scores.append(score)
    
    return np.mean(scores)

## trail Model

In [14]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold,train_test_split,cross_val_score

from sklearn.metrics import classification_report
import time

In [7]:
mlp = MLPClassifier()
knn = KNeighborsClassifier()
svc = SVC(probability=True)
dtc = DecisionTreeClassifier()
rfc = RandomForestClassifier()
abc = AdaBoostClassifier()
nbc = GaussianNB()
lrc = LogisticRegression(C = 1.0, penalty='l2', multi_class='ovr')
# gpc = GaussianProcessClassifier()

In [8]:
print(xtrain.shape)
print(ytrain.shape)

(213451, 312)
(213451, 1)


In [10]:
n = int(xtrain.shape[0]*0.03)
# id_test = test['id']

xtrain2 = xtrain.iloc[:n,:]
ytrain2 = ytrain_le[:n]
xtrain2 = StandardScaler().fit_transform(xtrain2)

kf = KFold(n_splits=5, random_state=2017)

for train_index,test_index in kf.split(xtrain2,ytrain2):
    x_train,x_test = xtrain2[train_index,:],xtrain2[test_index,:]
    y_train,y_test = ytrain2[train_index],ytrain2[test_index]

mlp.fit(x_train,y_train)
knn.fit(x_train,y_train)
svc.fit(x_train,y_train)
dtc.fit(x_train,y_train)
rfc.fit(x_train,y_train)
abc.fit(x_train,y_train)
nbc.fit(x_train,y_train)
lrc.fit(x_train,y_train)    
    
train_score_3size = []
cv_score_3size = []

allmodel_train_score_3size = []
allmodel_cv_score_3size = []
# select a k:
k_ndcg = 5

model_list = [mlp,knn,svc,dtc,rfc,abc,nbc,lrc]
for each_model in model_list:
    time1 = time.time()
    y_pred = each_model.predict(x_test)
    train_ndcg_score = ndcg_score(y_train, each_model.predict_proba(x_train), k = k_ndcg)
    cv_ndcg_score = ndcg_score(y_test, each_model.predict_proba(x_test), k=k_ndcg)
    train_score_3size.append(train_ndcg_score)
    cv_score_3size.append(cv_ndcg_score)
    time2 = time.time()
    print(each_model)
    print(classification_report(y_test,y_pred))   
    print ("\nThe training score is: {}".format(np.mean(train_score_3size)))
    print ("\nThe cv score is: {}".format(np.mean(cv_score_3size)))
    print('time cost =',time2-time1)
    print('====================================')
    
    allmodel_train_score_3size.append(np.mean(train_score_3size))
    allmodel_cv_score_3size.append(np.mean(cv_score_3size))

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)
             precision    recall  f1-score   support

          0       0.00      0.00      0.00         6
          1       0.06      0.04      0.05        23
          2       0.00      0.00      0.00        13
          3       0.03      0.09      0.04        33
          4       0.05      0.03      0.04        72
          5       0.00      0.00      0.00        23
          6       0.05      0.03      0.03        38
          7       0.47      0.59      0.53       491
          8       0.00      0.00      0.00        11
          9       0.00      0.00   

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
             precision    recall  f1-score   support

          0       0.00      0.00      0.00         6
          1       0.00      0.00      0.00        23
          2       0.00      0.00      0.00        13
          3       0.00      0.00      0.00        33
          4       0.00      0.00      0.00        72
          5       0.00      0.00      0.00        23
          6       0.00      0.00      0.00        38
          7       0.52      0.63      0.57       491
          8       0.00      0.00      0.00        11
          9       0.00      0.00      0.00         2
         10       0.47      0.64      0.54       501
         11       0.00      0.00      0.00        67

avg / total       0.38      0.49      0.43      1280


The training score is: 0.88170

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
             precision    recall  f1-score   support

          0       0.00      0.00      0.00         6
          1       0.00      0.00      0.00        23
          2       0.00      0.00      0.00        13
          3       0.04      0.06      0.05        33
          4       0.09      0.06      0.07        72
          5       0.00      0.00      0.00        23
          6       0.05      0.03      0.03        38
          7       0.44      0.43      0.43       491
          8       0.00      0.00      0.00        11
          9       0.00      0.00      0.00         2
         10       0.41      0.46      0.43       501
         11  

...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


GaussianNB(priors=None)
             precision    recall  f1-score   support

          0       0.01      0.17      0.02         6
          1       0.04      0.52      0.07        23
          2       0.01      0.54      0.02        13
          3       0.13      0.06      0.08        33
          4       0.05      0.01      0.02        72
          5       0.03      0.13      0.05        23
          6       0.04      0.05      0.05        38
          7       0.57      0.01      0.02       491
          8       0.02      0.09      0.03        11
          9       0.00      0.00      0.00         2
         10       0.00      0.00      0.00       501
         11       0.00      0.00      0.00        67

avg / total       0.23      0.03      0.01      1280


The training score is: 0.7840200235946817

The cv score is: 0.5825381224363143
time cost = 0.8494765758514404


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
             precision    recall  f1-score   support

          0       0.02      0.33      0.04         6
          1       0.00      0.00      0.00        23
          2       0.01      0.23      0.02        13
          3       0.00      0.00      0.00        33
          4       0.00      0.00      0.00        72
          5       0.00      0.00      0.00        23
          6       0.00      0.00      0.00        38
          7       0.57      0.50      0.53       491
          8       0.00      0.00      0.00        11
          9       0.00      0.00      0.00         2
         10       0.45      0.43      0.44       501
         11       0.00      0.00      0.00        67

avg / total       0.39      0.36      0.38  

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [11]:
n = int(xtrain.shape[0]*0.05)
# id_test = test['id']

xtrain3 = xtrain.iloc[:n,:]
ytrain3 = ytrain_le[:n]

xtrain3 = StandardScaler().fit_transform(xtrain3)


kf = KFold(n_splits=5, random_state=2017)

for train_index,test_index in kf.split(xtrain3,ytrain3):
    x_train,x_test = xtrain3[train_index,:],xtrain3[test_index,:]
    y_train,y_test = ytrain3[train_index],ytrain3[test_index]


    
mlp.fit(x_train,y_train)
knn.fit(x_train,y_train)
svc.fit(x_train,y_train)
dtc.fit(x_train,y_train)
rfc.fit(x_train,y_train)
abc.fit(x_train,y_train)
nbc.fit(x_train,y_train)
lrc.fit(x_train,y_train)    
    
train_score_5size = []
cv_score_5size = []

allmodel_train_score_5size = []
allmodel_cv_score_5size = []
# select a k:
k_ndcg = 5


model_list = [mlp,knn,svc,dtc,rfc,abc,nbc,lrc]
for each_model in model_list:
    time1 = time.time()
    y_pred = each_model.predict(x_test)
    train_ndcg_score = ndcg_score(y_train, each_model.predict_proba(x_train), k = k_ndcg)
    cv_ndcg_score = ndcg_score(y_test, each_model.predict_proba(x_test), k=k_ndcg)
    train_score_5size.append(train_ndcg_score)
    cv_score_5size.append(cv_ndcg_score)
    time2 = time.time()
    print(each_model)
    print(classification_report(y_test,y_pred))   
    print ("\nThe training score is: {}".format(np.mean(train_score_5size)))
    print ("\nThe cv score is: {}".format(np.mean(cv_score_5size)))
    print('time cost =',time2-time1)
    print('====================================')
    
    allmodel_train_score_5size.append(np.mean(train_score_5size))
    allmodel_cv_score_5size.append(np.mean(cv_score_5size))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)
             precision    recall  f1-score   support

          0       0.00      0.00      0.00         4
          1       0.00      0.00      0.00        23
          2       0.00      0.00      0.00        17
          3       0.00      0.00      0.00        32
          4       0.03      0.03      0.03        78
          5       0.00      0.00      0.00        26
          6       0.00      0.00      0.00        28
          7       0.71      0.30      0.42      1134
          8       0.00      0.00      0.00        10
          9       0.00      0.00   

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')
             precision    recall  f1-score   support

          0       0.00      0.00      0.00         4
          1       0.00      0.00      0.00        23
          2       0.12      0.06      0.08        17
          3       0.00      0.00      0.00        32
          4       0.10      0.06      0.08        78
          5       0.00      0.00      0.00        26
          6       0.00      0.00      0.00        28
          7       0.59      0.52      0.55      1134
          8       0.00      0.00      0.00        10
          9       0.00      0.00      0.00         4
         10       0.38      0.54      0.45       712
         11       0.00      0.00      0.00        66

avg / total       0.44      0.46      0.45      2134


The training score is: 0.896821661196062

The cv score is: 0.6875123405360498
time cost = 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


GaussianNB(priors=None)
             precision    recall  f1-score   support

          0       0.00      0.00      0.00         4
          1       0.02      0.04      0.03        23
          2       0.01      0.06      0.01        17
          3       0.07      0.03      0.04        32
          4       0.00      0.00      0.00        78
          5       0.00      0.04      0.01        26
          6       0.00      0.00      0.00        28
          7       0.87      0.02      0.05      1134
          8       0.00      0.60      0.01        10
          9       0.00      0.00      0.00         4
         10       0.17      0.00      0.00       712
         11       0.01      0.02      0.01        66

avg / total       0.52      0.02      0.03      2134


The training score is: 0.7736805197711641

The cv score is: 0.6216014797474997
time cost = 1.1965413093566895


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
             precision    recall  f1-score   support

          0       0.00      0.00      0.00         4
          1       0.00      0.00      0.00        23
          2       0.00      0.00      0.00        17
          3       0.00      0.00      0.00        32
          4       0.00      0.00      0.00        78
          5       0.00      0.00      0.00        26
          6       0.00      0.00      0.00        28
          7       0.70      0.48      0.57      1134
          8       0.00      0.00      0.00        10
          9       0.00      0.00      0.00         4
         10       0.40      0.76      0.53       712
         11       0.00      0.00      0.00        66

avg / total       0.51      0.51      0.48  

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [13]:
n = int(xtrain.shape[0]*0.1)
# id_test = test['id']

xtrain4 = xtrain.iloc[:n,:]
ytrain4 = ytrain_le[:n]

xtrain4 = StandardScaler().fit_transform(xtrain4)


kf = KFold(n_splits=5, random_state=2017)

for train_index,test_index in kf.split(xtrain4,ytrain4):
    x_train,x_test = xtrain4[train_index,:],xtrain4[test_index,:]
    y_train,y_test = ytrain4[train_index],ytrain4[test_index]


    
mlp.fit(x_train,y_train)
knn.fit(x_train,y_train)
svc.fit(x_train,y_train)
dtc.fit(x_train,y_train)
rfc.fit(x_train,y_train)
abc.fit(x_train,y_train)
nbc.fit(x_train,y_train)
lrc.fit(x_train,y_train)    
    
train_score_10size = []
cv_score_10size = []

allmodel_train_score_10size = []
allmodel_cv_score_10size = []
# select a k:
k_ndcg = 5


model_list = [mlp,knn,svc,dtc,rfc,abc,nbc,lrc]
for each_model in model_list:
    time1 = time.time()
    y_pred = each_model.predict(x_test)
    train_ndcg_score = ndcg_score(y_train, each_model.predict_proba(x_train), k = k_ndcg)
    cv_ndcg_score = ndcg_score(y_test, each_model.predict_proba(x_test), k=k_ndcg)
    train_score_10size.append(train_ndcg_score)
    cv_score_10size.append(cv_ndcg_score)
    time2 = time.time()
    print(each_model)
    print(classification_report(y_test,y_pred))   
    print ("\nThe training score is: {}".format(np.mean(train_score_10size)))
    print ("\nThe cv score is: {}".format(np.mean(cv_score_10size)))
    print('time cost =',time2-time1)
    print('====================================')
    
    allmodel_train_score_10size.append(np.mean(train_score_10size))
    allmodel_cv_score_10size.append(np.mean(cv_score_10size))

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)
             precision    recall  f1-score   support

          0       0.00      0.00      0.00         5
          1       0.00      0.00      0.00        37
          2       0.00      0.00      0.00        36
          3       0.07      0.03      0.04        75
          4       0.04      0.12      0.05       156
          5       0.07      0.04      0.05        78
          6       0.04      0.05      0.04        98
          7       0.61      0.48      0.54      2022
          8       0.00      0.00      0.00        19
          9       0.00      0.00   

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')
             precision    recall  f1-score   support

          0       0.00      0.00      0.00         5
          1       0.00      0.00      0.00        37
          2       0.00      0.00      0.00        36
          3       0.00      0.00      0.00        75
          4       0.04      0.02      0.03       156
          5       0.00      0.00      0.00        78
          6       0.00      0.00      0.00        98
          7       0.54      0.61      0.57      2022
          8       0.00      0.00      0.00        19
          9       0.00      0.00      0.00         6
         10       0.41      0.49      0.45      1523
         11       0.06      0.00      0.01       214

avg / total       0.41      0.46      0.43      4269


The training score is: 0.8622469131512822

The cv score is: 0.6844021004932097
time cost =

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
             precision    recall  f1-score   support

          0       0.00      0.00      0.00         5
          1       0.00      0.00      0.00        37
          2       0.00      0.00      0.00        36
          3       0.00      0.00      0.00        75
          4       0.00      0.00      0.00       156
          5       0.00      0.00      0.00        78
          6       0.00      0.00      0.00        98
          7       0.61      0.67      0.64      2022
          8       0.00      0.00      0.00        19
          9       0.00      0.00      0.00         6
         10       0.46      0.62      0.53      1523
         11       0.00      0.00      0.00       214

avg / total       0.45      0.54      0.49      4269


The training score is: 0.84878

...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None)
             precision    recall  f1-score   support

          0       0.00      0.00      0.00         5
          1       0.00      0.00      0.00        37
          2       0.00      0.00      0.00        36
          3       0.00      0.00      0.00        75
          4       0.00      0.00      0.00       156
          5       0.00      0.00      0.00        78
          6       0.00      0.00      0.00        98
          7       0.62      0.33      0.43      2022
          8       0.00      0.00      0.00        19
          9       0.00      0.00      0.00         6
         10       0.39      0.74      0.51      1523
         11       0.00      0.00      0.00       214

avg / total       0.43      0.42      0.38      4269


The training score is: 0.8814953535870851

The cv score is: 0.6948797556027216
time cost = 1.9006662368774414


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


GaussianNB(priors=None)
             precision    recall  f1-score   support

          0       0.00      0.40      0.01         5
          1       0.00      0.05      0.01        37
          2       0.01      0.31      0.02        36
          3       0.00      0.00      0.00        75
          4       0.00      0.00      0.00       156
          5       0.00      0.01      0.01        78
          6       0.03      0.01      0.01        98
          7       0.59      0.01      0.01      2022
          8       0.00      0.26      0.01        19
          9       0.00      0.00      0.00         6
         10       0.60      0.00      0.00      1523
         11       0.00      0.00      0.00       214

avg / total       0.49      0.01      0.01      4269


The training score is: 0.7629238620785114

The cv score is: 0.6048880845327643
time cost = 2.459460735321045
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100,

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [21]:
allmodel_dvalue_10size = list(map(lambda x,y:x - y,allmodel_train_score_10size,allmodel_cv_score_10size))
allmodel_dvalue_5size = list(map(lambda x,y:x - y,allmodel_train_score_5size,allmodel_cv_score_5size))
allmodel_dvalue_3size = list(map(lambda x,y:x - y,allmodel_train_score_3size,allmodel_cv_score_3size))

In [24]:
allmodel_score = pd.DataFrame([allmodel_train_score_10size,allmodel_cv_score_10size,allmodel_dvalue_10size,
                               allmodel_train_score_5size,allmodel_cv_score_5size,allmodel_dvalue_5size,
                               allmodel_train_score_3size,allmodel_cv_score_3size,allmodel_dvalue_3size,],
                             index=['trainscore_10size','cvscore_10size','dvalue_10size',
                                    'trainscore_5size','cvscore_5size','dvalue_5size',
                                    'trainscore_3size','cvscore_3size','dvalue_3size'],
                             columns=['MLP','KNN','SVC','DTC',
                                      'RFC','ABC','GNB','LRC'])
allmodel_score

,MLP,KNN,SVC,DTC,RFC,ABC,GNB,LRC
trainscore_10size,0.885344,0.862247,0.848789,0.885178,0.905543,0.881495,0.762924,0.766115
cvscore_10size,0.673796,0.684402,0.713058,0.681983,0.689968,0.694880,0.604888,0.625078
dvalue_10size,0.211548,0.177845,0.135731,0.203196,0.215575,0.186616,0.158036,0.141037
trainscore_5size,0.963352,0.896822,0.868504,0.899949,0.917290,0.887737,0.773681,0.774273
cvscore_5size,0.680105,0.687512,0.713761,0.696131,0.705807,0.710520,0.621601,0.638895
dvalue_5size,0.283247,0.209309,0.154743,0.203818,0.211482,0.177217,0.152079,0.135378
trainscore_3size,0.992668,0.912522,0.881707,0.910019,0.925607,0.896920,0.784020,0.785273
cvscore_3size,0.607008,0.636387,0.668316,0.642102,0.651192,0.661854,0.582538,0.588094
dvalue_3size,0.385660,0.276135,0.213391,0.267916,0.274416,0.235066,0.201482,0.197179


## Use SVC model

In [8]:
kernel_list = ['linear', 'poly', 'rbf', 'sigmoid']
C_list = [0.5,1.0,1.5,2.0,3.0]
coef0_list = [0,1,2,4,8]

### Change kernel

In [15]:
n = int(xtrain.shape[0]*0.05)
# id_test = test['id']
xtrain5 = xtrain.iloc[:n,:]
ytrain5 = ytrain_le[:n]
xtrain5 = StandardScaler().fit_transform(xtrain5)

train_score = []
cv_score = []
allkernel_train_score_5size=[]
allkernel_cv_score_5size=[]

k_ndcg = 5

for train_index,test_index in kf.split(xtrain5,ytrain5):
    x_train,x_test = xtrain5[train_index,:],xtrain5[test_index,:]
    y_train,y_test = ytrain5[train_index],ytrain5[test_index]

for kernel_index in kernel_list:
    print('kernel {}:'.format(kernel_index))
    svc = SVC(kernel=str(kernel_index),probability=True)
    print(svc)
    
    train_score_iter = []
    cv_score_iter = []
    
    time1 = time.time()
    svc.fit(x_train,y_train)
    train_ndcg_score = ndcg_score(y_train, svc.predict_proba(x_train), k = k_ndcg)
    cv_ndcg_score = ndcg_score(y_test, svc.predict_proba(x_test), k=k_ndcg)
    train_score_iter.append(train_ndcg_score)
    cv_score_iter.append(cv_ndcg_score)
    time2 = time.time()
    
    train_score.append(np.mean(train_score_iter))
    cv_score.append(np.mean(cv_score_iter))
    
    print ("\nThe training score is: {}".format(np.mean(train_score_iter)))
    print ("The cv score is: {}\n".format(np.mean(cv_score_iter)))
    print("timecost is: ",time2-time1)
    
    allkernel_train_score_5size.append(np.mean(train_score_iter))
    allkernel_cv_score_5size.append(np.mean(cv_score_iter))

kernel linear:
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

The training score is: 0.7703691541036612
The cv score is: 0.7651392095003176

timecost is:  599.3721969127655
kernel poly:
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='poly',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

The training score is: 0.798550036387934
The cv score is: 0.7412000533989783

timecost is:  346.5637946128845
kernel rbf:
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

The training score is: 0.8114614355806535
The cv score is: 0

In [16]:
n = int(xtrain.shape[0]*0.1)
# id_test = test['id']

xtrain6 = xtrain.iloc[:n,:]
ytrain6 = ytrain_le[:n]

xtrain6 = StandardScaler().fit_transform(xtrain6)

train_score = []
cv_score = []

allkernel_train_score_10size=[]
allkernel_cv_score_10size=[]

k_ndcg = 5

for train_index,test_index in kf.split(xtrain6,ytrain6):
    x_train,x_test = xtrain6[train_index,:],xtrain6[test_index,:]
    y_train,y_test = ytrain6[train_index],ytrain6[test_index]

for kernel_index in kernel_list:
    print('kernel {}:'.format(kernel_index))
    svc = SVC(kernel=str(kernel_index),probability=True)
    print(svc)
    
    train_score_iter = []
    cv_score_iter = []
    
    time1 = time.time()
    svc.fit(x_train,y_train)
    train_ndcg_score = ndcg_score(y_train, svc.predict_proba(x_train), k = k_ndcg)
    cv_ndcg_score = ndcg_score(y_test, svc.predict_proba(x_test), k=k_ndcg)
    train_score_iter.append(train_ndcg_score)
    cv_score_iter.append(cv_ndcg_score)
    time2 = time.time()
    
    train_score.append(np.mean(train_score_iter))
    cv_score.append(np.mean(cv_score_iter))
    
    print ("\nThe training score is: {}".format(np.mean(train_score_iter)))
    print ("The cv score is: {}\n".format(np.mean(cv_score_iter)))
    print("timecost is: ",time2-time1)
    
    allkernel_train_score_10size.append(np.mean(train_score_iter))
    allkernel_cv_score_10size.append(np.mean(cv_score_iter))

kernel linear:
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

The training score is: 0.7827245145701719
The cv score is: 0.7725023733170951

timecost is:  3505.029576778412
kernel poly:
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='poly',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

The training score is: 0.8101949951508604
The cv score is: 0.7586636608321573

timecost is:  1352.5243601799011
kernel rbf:
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

The training score is: 0.8219314272424414
The cv score is:

In [27]:
allkernel_dvalue_10size = list(map(lambda x,y : x-y,allkernel_train_score_10size,allkernel_cv_score_10size))
allkernel_dvalue_5size = list(map(lambda x,y : x-y,allkernel_train_score_5size,allkernel_cv_score_5size))

allkernel_score = pd.DataFrame([allkernel_train_score_10size,allkernel_cv_score_10size,allkernel_dvalue_10size,
                                allkernel_train_score_5size,allkernel_cv_score_5size,allkernel_dvalue_5size],
                               index=['trainscore_10size','cvscore_10size','dvalue_10size',
                                      'trainscore_5size','cvscore_5size','dvalue_5size'],
                               columns=['linear_kernel', 'poly_kernel', 'rbf_kernel', 'sigmoid_kernel'])
allkernel_score

,linear_kernel,poly_kernel,rbf_kernel,sigmoid_kernel
trainscore_10size,0.782725,0.810195,0.821931,0.768617
cvscore_10size,0.772502,0.758664,0.770585,0.771367
dvalue_10size,0.010222,0.051531,0.051346,-0.002750
trainscore_5size,0.770369,0.798550,0.811461,0.754002
cvscore_5size,0.765139,0.741200,0.765937,0.766235
dvalue_5size,0.005230,0.057350,0.045525,-0.012233


### Change C

In [7]:
SVC?

In [ ]:
n = int(xtrain.shape[0]*0.05)
# id_test = test['id']

xtrain7 = xtrain.iloc[:n,:]
ytrain7 = ytrain_le[:n]

xtrain7 = StandardScaler().fit_transform(xtrain7)
kf = KFold(n_splits=5, random_state=2017)

train_score = []
cv_score = []

parameterC_train_score = []
parameterC_cv_score = []

k_ndcg = 5

for C_index in C_list:
    print('C {}:'.format(C_index))
    svc = SVC(kernel='linear',probability=True,C=C_index)
    print(svc)
    
    train_score_iter = []
    cv_score_iter = []
    
    for train_index,test_index in kf.split(xtrain7,ytrain7):
        x_train,x_test = xtrain7[train_index,:],xtrain7[test_index,:]
        y_train,y_test = ytrain7[train_index],ytrain7[test_index]
        
        time1=time.time()
        svc.fit(x_train,y_train)
                
        train_ndcg_score = ndcg_score(y_train, svc.predict_proba(x_train), k = k_ndcg)
        cv_ndcg_score = ndcg_score(y_test, svc.predict_proba(x_test), k=k_ndcg)
        train_score_iter.append(train_ndcg_score)
        cv_score_iter.append(cv_ndcg_score)
        
        time2=time.time()
        
    train_score.append(np.mean(train_score_iter))
    cv_score.append(np.mean(cv_score_iter))
    
    print ("\nThe training score is: {}".format(np.mean(train_score_iter)))
    print ("The cv score is: {}\n".format(np.mean(cv_score_iter)))
    print('timecost is :' ,time2-time1)
    
    parameterC_train_score.append(np.mean(train_score_iter))
    parameterC_cv_score.append(np.mean(cv_score_iter))

C 0.5:
SVC(C=0.5, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

The training score is: 0.7761060426031391
The cv score is: 0.757432422165435

timecost is : 469.1493308544159
C 1.0:
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

The training score is: 0.7761636738883192
The cv score is: 0.7571905870137627

timecost is : 575.6648802757263
C 1.5:
SVC(C=1.5, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

The training score is: 0.7760566247991335
The cv score is: 0.7571612246996

In [15]:
n = int(xtrain.shape[0]*0.03)
# id_test = test['id']
xtrain5 = xtrain.iloc[:n,:]
ytrain5 = ytrain_le[:n]
xtrain5 = StandardScaler().fit_transform(xtrain5)
kf = KFold(n_splits=5, random_state=2017)
train_score = []
cv_score = []

k_ndcg = 5

for train_index,test_index in kf.split(xtrain5,ytrain5):
    x_train,x_test = xtrain5[train_index,:],xtrain5[test_index,:]
    y_train,y_test = ytrain5[train_index],ytrain5[test_index]


svc.fit(x_train,y_train)
y_pred = svc.predict_proba(x_test)

In [18]:
y_pred2 = svc.predict(x_test)

In [22]:
pd.DataFrame({"columns":list(xtrain.columns)[1:], "coef":list(svc.coef_.T)})

AttributeError: coef_ is only available when using a linear kernel

In [21]:
xtrain5

array([[  3.57714970e+01,  -8.80710901e-01,  -1.13364609e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  3.57714970e+01,  -8.80710901e-01,  -1.13364609e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  3.57714970e+01,  -8.80710901e-01,  -1.13364609e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       ..., 
       [ -2.79552181e-02,  -8.80710901e-01,   8.82109511e-01, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [ -2.79552181e-02,  -8.80710901e-01,   8.82109511e-01, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [ -2.79552181e-02,  -8.80710901e-01,   8.82109511e-01, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00]])